In [116]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

orderbook = "data/GOOG_2012-06-21_34200000_57600000_orderbook_10.csv"
message = "data/GOOG_2012-06-21_34200000_57600000_message_10.csv"
num_levels = 10

header_list = []
for i in range(num_levels):
    header_list = header_list + ["Pa%d"%(i+1),"Va%d"%(i+1),"Pb%d"%(i+1),"Vb%d"%(i+1)]
df_orderbook = pd.read_csv(orderbook,header=None,names=header_list)

df_message = pd.read_csv(message,usecols = [0,1,3,4,5], names=['time', 'type','size','price','direction'])
df_message.index = pd.Timestamp(datetime.date.today()) + pd.TimedeltaIndex(df_message.time, unit='s')
df_orderbook.index = df_message.index

def labelling(a):
    if a > 0:
        b = 1
    else:
        b= 0
    return b

# Spreads and mid-prices
def feature_v2(num_levels,df): # 20
    for i in range(1,num_levels+1):
        df["spread%d"%(i)] = df["Pa%d"%(i)] - df["Pb%d"%(i)]
        df["midprice%d"%(i)] = (df["Pa%d"%(i)] + df["Pb%d"%(i)])/2
    return df

def feature_v3(num_levels,df): # 20 - 2
    for i in range(1, num_levels):
        df["PA_diff%d"%(i)] = df["Pa%d"%(i+1)] - df["Pa%d"%(i)]
        df["PB_diff%d"%(i)] = df["Pb%d"%(i)] - df["Pb%d"%(i+1)]
    return df

def feature_v4(num_levels,df): # 4
    lst = ["Pa%d"%(i+1) for i in range(num_levels)]
    df["Pa_mean"] = df[df.columns.intersection(lst)].sum(axis=1)    
    
    lst = ["Pb%d"%(i+1) for i in range(num_levels)]
    df["Pb_mean"] = df[df.columns.intersection(lst)].sum(axis=1)
    
    lst = ["Va%d"%(i+1) for i in range(num_levels)]
    df["Va_mean"] = df[df.columns.intersection(lst)].sum(axis=1)
    
    lst = ["Vb%d"%(i+1) for i in range(num_levels)]
    df["Vb_mean"] = df[df.columns.intersection(lst)].sum(axis=1)
    return df

def feature_v5(num_levels,df): # 20
    for i in range(num_levels): #
        df["pri_accum_diff%d"%(i+1)] = 0
        df["vol_accum_diff%d"%(i+1)] = 0
        for k in range(i):
            df["pri_accum_diff%d"%(i+1)] += (df["Pa%d"%(k+1)] - df["Pb%d"%(k+1)])
            df["vol_accum_diff%d"%(i+1)] += (df["Va%d"%(i+1)] - df["Vb%d"%(i+1)])
    return df

def normalize_input(X_train, X_test):
    scaler = StandardScaler()
    # Fit on training set only.
    scaler.fit(X_train)
    # Apply transform to both the training set and the test set.
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)
    return X_train, X_test

df_orderbook = feature_v2(num_levels,df_orderbook)
df_orderbook = feature_v3(num_levels,df_orderbook)
df_orderbook = feature_v4(num_levels,df_orderbook)
df_orderbook = feature_v5(num_levels,df_orderbook)

# df_orderbook.head()

# window = 1000
# df_orderbook['midprice_win%d'%window] = df_orderbook.rolling(window).agg({'midprice1':'mean'})
# df_orderbook['midprice_win%d'%window].plot()

# window = '1MIN'
# df_orderbook['midprice_average'+window] = df_orderbook.rolling('1MIN').agg({'midprice1':'mean'})
# df_orderbook['midprice_average'+window].plot()
# plt.show()

df_orderbook1s = df_orderbook.resample('1S').first()
df_orderbook1s["price_10min"] = df_orderbook1s['midprice1'].shift(-600)
df_orderbook1s.dropna(inplace=True)
df_orderbook1s["price_change"] = df_orderbook1s["price_10min"] - df_orderbook1s['midprice1']

In [117]:
X = df_orderbook1s.drop(['price_10min','price_change'], axis=1).values
y = list(map(labelling, df_orderbook1s["price_change"]))

In [30]:
from sklearn.metrics import precision_score, accuracy_score, recall_score, f1_score
def model_scoring(y_test, y_pred):
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    print("======================================")
    print("======Model Performance===============")

    print("the accuracy is: ", accuracy)
    print("the precision is：",precision)
    print("the recall is: ", recall)
    print("the f1 score is: ", f1)

In [97]:
# logistic regression
from sklearn.linear_model import LogisticRegression
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_test = normalize_input(X_train, X_test)
clf = LogisticRegression(random_state=0, 
                         penalty='l2',
                         solver='lbfgs',
                         max_iter = 100000,
                         multi_class='ovr').fit(X_train, y_train)
print("training score.", clf.score(X_train, y_train))
print("testing score.", clf.score(X_test, y_test))
from sklearn.metrics import confusion_matrix
y_pred = clf.predict(X_test)
confusion_matrix(y_test, y_predict)
model_scoring(y_test, y_pred)

training score. 0.690598091691878
testing score. 0.6886924150767799
======Model Performance===============
the accuracy is:  0.6886924150767799
the precision is： 0.6120689655172413
the recall is:  0.10215827338129496
the f1 score is:  0.1750924784217016


In [100]:
############################
 ######### XGBoost ########
############################
import xgboost as xgb
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_test = normalize_input(X_train, X_test)
xgb_clf = xgb.XGBClassifier()
xgb_clf.fit(X_train, y_train)
y_pred = xgb_clf.predict(X_test)
model_scoring(y_test, y_pred)

======Model Performance===============
the accuracy is:  0.8245695672405771
the precision is： 0.8057692307692308
the recall is:  0.6028776978417266
the f1 score is:  0.6897119341563785


In [146]:
############################
############ CNN ###########
############################
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv1D,  MaxPooling1D
import pickle
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_test = normalize_input(X_train, X_test)
model = Sequential()
model.add(Conv1D(256, 4, input_shape=(102, 1)))
model.add(Activation('relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(256, 2))
model.add(Activation('relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64))
model.add(Dense(1))
model.add(Activation('sigmoid'))
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
# model.summary()
model.fit(X_train.reshape((X_train.shape[0],X_train.shape[1],1)), np.asarray(y_train), 
          batch_size=32, epochs=30, validation_split=0.3)
y_pred = model.predict(X_test.reshape((X_test.shape[0],X_test.shape[1],1)))
y_pred[y_pred>0.5] = 1
y_pred[y_pred<0.5] = 0
model_scoring(y_test, y_pred.reshape((-1,)))

Train on 6015 samples, validate on 2579 samples
Epoch 1/30
6015/6015 [==============================] - 3s 510us/sample - loss: 0.6169 - accuracy: 0.6806 - val_loss: 0.6074 - val_accuracy: 0.6937
Epoch 2/30
6015/6015 [==============================] - 2s 396us/sample - loss: 0.5782 - accuracy: 0.6974 - val_loss: 0.5781 - val_accuracy: 0.7022
Epoch 3/30
6015/6015 [==============================] - 2s 390us/sample - loss: 0.5594 - accuracy: 0.7142 - val_loss: 0.5659 - val_accuracy: 0.7107
Epoch 4/30
6015/6015 [==============================] - 2s 388us/sample - loss: 0.5398 - accuracy: 0.7330 - val_loss: 0.5478 - val_accuracy: 0.7297
Epoch 5/30
6015/6015 [==============================] - 2s 394us/sample - loss: 0.5137 - accuracy: 0.7493 - val_loss: 0.5405 - val_accuracy: 0.7255
Epoch 6/30
6015/6015 [==============================] - 2s 390us/sample - loss: 0.4964 - accuracy: 0.7568 - val_loss: 0.5450 - val_accuracy: 0.7332
Epoch 7/30
6015/6015 [==============================] - 2s 393us

In [ ]:
# Convert y2 to dummy variables
y2 = np.zeros((y.shape[0], max_features), dtype=np.float32)
y2[np.arange(y.shape[0]), y] = 1.0
print(y2)


print('Build model...')
model = Sequential()
model.add(Conv1D(128, kernel_size=x.shape[1], input_shape=(None, 1)))
model.add(Dense(4, activation='sigmoid'))

# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()
print('Train...')

In [105]:
print(x)

[[[0.]
  [1.]
  [1.]
  [0.]
  [0.]
  [0.]]

 [[0.]
  [0.]
  [0.]
  [2.]
  [2.]
  [0.]]

 [[0.]
  [0.]
  [0.]
  [0.]
  [3.]
  [3.]]

 [[0.]
  [2.]
  [2.]
  [0.]
  [0.]
  [0.]]

 [[0.]
  [0.]
  [3.]
  [3.]
  [0.]
  [0.]]

 [[0.]
  [0.]
  [0.]
  [0.]
  [1.]
  [1.]]]


In [83]:
model.fit(x,y2,epochs=200)
pred = model.predict(x)
predict_classes = np.argmax(pred,axis=1)
print("Predicted classes: {}",predict_classes)
print("Expected classes: {}",predict_classes)

[[0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]
 [0. 1. 0. 0.]]
Build model...
Train...
Train on 6 samples
Epoch 1/200
6/6 [==============================] - 0s 7ms/sample


ValueError: logits and labels must have the same shape ((None, 1, 4) vs (None, 4))

In [74]:
y_train

[0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,


In [34]:
####################
#### Light gbm ####
####################

======Model Performance===============
the accuracy is:  0.6886924150767799
the precision is： 0.6120689655172413
the recall is:  0.10215827338129496
the f1 score is:  0.1750924784217016


In [ ]:
# CNN + Fully Connected layer
# RNN example and LSTM example
40 + 20 + 
X_test.shape


In [23]:
# Counter(best_preds)
Counter(y_test)

Counter({1: 695, 0: 1454})

In [4]:
import xgboost

In [ ]:
########################
# Support Vector machine
########################
###  https://towardsdatascience.com/preprocessing-with-sklearn-a-complete-and-comprehensive-guide-670cb98fcfb9
from sklearn import svm
import time

########## Training ##########
clf = svm.SVC(kernel='linear') # Linear Kernel
time1 = time.time()
clf.fit(X_train, y_train)
print(time.time() - time1)

########## Testing ##########
y_pred = clf.predict(X_test)

from sklearn import metrics
# Model Accuracy: how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
# Model Precision: what percentage of positive tuples are labeled as such?
print("Precision:",metrics.precision_score(y_test, y_pred))
# Model Recall: what percentage of positive tuples are labelled as such?
print("Recall:",metrics.recall_score(y_test, y_pred))

In [ ]:
import timeit
import warnings

import matplotlib.pyplot as plt
import numpy as np

from sklearn.datasets import fetch_20newsgroups_vectorized
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.exceptions import ConvergenceWarning

In [ ]:
print(__doc__)
# Author: Arthur Mensch

warnings.filterwarnings("ignore", category=ConvergenceWarning,
                        module="sklearn")
t0 = timeit.default_timer()

# We use SAGA solver
solver = 'saga'

# Turn down for faster run time
n_samples = 10000

# Memorized fetch_rcv1 for faster access
X, y = fetch_20newsgroups_vectorized('all', return_X_y=True)

In [ ]:
X = X[:n_samples]
y = y[:n_samples]

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    random_state=42,
                                                    stratify=y,
                                                    test_size=0.1)
train_samples, n_features = X_train.shape
n_classes = np.unique(y).shape[0]

print('Dataset 20newsgroup, train_samples=%i, n_features=%i, n_classes=%i'
      % (train_samples, n_features, n_classes))

models = {'ovr': {'name': 'One versus Rest', 'iters': [1, 2, 4]},
          'multinomial': {'name': 'Multinomial', 'iters': [1, 3, 7]}}

for model in models:
    # Add initial chance-level values for plotting purpose
    accuracies = [1 / n_classes]
    times = [0]
    densities = [1]

    model_params = models[model]

    # Small number of epochs for fast runtime
    for this_max_iter in model_params['iters']:
        print('[model=%s, solver=%s] Number of epochs: %s' %
              (model_params['name'], solver, this_max_iter))
        lr = LogisticRegression(solver=solver,
                                multi_class=model,
                                penalty='l1',
                                max_iter=this_max_iter,
                                random_state=42,
                                )
        t1 = timeit.default_timer()
        lr.fit(X_train, y_train)
        
        train_time = timeit.default_timer() - t1

        y_pred = lr.predict(X_test)
        accuracy = np.sum(y_pred == y_test) / y_test.shape[0]
        density = np.mean(lr.coef_ != 0, axis=1) * 100
        accuracies.append(accuracy)
        densities.append(density)
        times.append(train_time)
    models[model]['times'] = times
    models[model]['densities'] = densities
    models[model]['accuracies'] = accuracies
    print('Test accuracy for model %s: %.4f' % (model, accuracies[-1]))
    print('%% non-zero coefficients for model %s, '
          'per class:\n %s' % (model, densities[-1]))
    print('Run time (%i epochs) for model %s:'
          '%.2f' % (model_params['iters'][-1], model, times[-1]))

fig = plt.figure()
ax = fig.add_subplot(111)

for model in models:
    name = models[model]['name']
    times = models[model]['times']
    accuracies = models[model]['accuracies']
    ax.plot(times, accuracies, marker='o',
            label='Model: %s' % name)
    ax.set_xlabel('Train time (s)')
    ax.set_ylabel('Test accuracy')
ax.legend()
fig.suptitle('Multinomial vs One-vs-Rest Logistic L1\n'
             'Dataset %s' % '20newsgroups')
fig.tight_layout()
fig.subplots_adjust(top=0.85)
run_time = timeit.default_timer() - t0
print('Example run in %.3f s' % run_time)
plt.show()

In [ ]:
from sklearn.datasets import fetch_mldata
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import pandas as pd
import numpy as np
mnist = fetch_mldata('MNIST original')

In [ ]:
print(mnist.data.shape)
print(mnist.COL_NAMES)
print(mnist.target.shape)
print(np.unique(mnist.target))

In [ ]:
train_img, test_img, train_lbl, test_lbl = train_test_split(
    mnist.data, mnist.target, test_size=1/7.0, random_state=122)

  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1316 sha256=cfe2316df0888dd568a5dec1988dc30ad625e9252a540894849586e385185676
  Stored in directory: /home/zhangge/.cache/pip/wheels/76/03/bb/589d421d27431bcd2c6da284d5f2286c8e3b2ea3cf1594c074
Successfully built sklearn
ERROR: Could not install packages due to an EnvironmentError: [Errno 13] Permission denied: '/usr/local/lib/python3.6/dist-packages/sklearn-0.0.dist-info'
Consider using the `--user` option or check the permissions.

